In [1]:
import pandas as pd
import numpy as np
import io

In [2]:
fifa = pd.read_csv('players_22.csv')

/tmp/ipykernel_31283/2373166978.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv('players_22.csv')


In [3]:
# Printing the first 5 entries in the dataset 
# to see how the dataset looks like
fifa.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [4]:
# Finding the percentage of missing values in every column
# and dropping the column that are more than 60% empty
def missingData(data):
    # Handling missing values
    col = []
    for c in data.columns:
        missingVal = np.mean(data[c].isnull())*100
        if missingVal > 60:
            print('{} - {}%'.format(c, round(missingVal)))
            col.append(c)
    # Droping 60%+ empty columns
    data.drop(columns = col ,inplace = True, axis=0)
    
    #Printing the dropped columns
    print('Columns dropped :  \n\n',col) 
            

In [5]:
# Calling the missingaData function 
# and passing our dataframe to it
missingData(fifa)

club_loaned_from - 94%
nation_team_id - 96%
nation_position - 96%
nation_jersey_number - 96%
player_tags - 93%
goalkeeping_speed - 89%
nation_logo_url - 96%
Columns dropped :  

 ['club_loaned_from', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'player_tags', 'goalkeeping_speed', 'nation_logo_url']


In [6]:
# Printing the structure of our dataframe 
# after dropping 60% + empty column
print("After handling the missing values, the dataset has : ",fifa.shape[0],"rows and " ,fifa.shape[1],"columns")

After handling the missing values, the dataset has :  19239 rows and  103 columns


In [7]:
# Imputing the dataframe
def replace_missing_values(data1):
    # Get the column names by data types
    col_names = data1.columns
    col_types = data1.dtypes

    # Loop through each column and replace missing values
    for i in range(len(col_names)):
        col_name = col_names[i]
        col_type = col_types[i]
        
        if col_type == 'object':
            # For string columns, use bfill and ffill methods
            data1[col_name] = data1[col_name].fillna(method='bfill').fillna(method='ffill')
        else:
            # For numerical columns, replace missing values with the mean of the column
            data1[col_name] = data1[col_name].fillna(fifa[col_name].mean())
    
    


In [8]:
# Imputing all the missing cells
replace_missing_values(fifa)


In [9]:
# Get the count of missing values per row
missing_per_row = fifa.isnull().sum(axis=1)

# Get the total number of rows with missing values
total_missing_rows = len(missing_per_row[missing_per_row > 0])

print(total_missing_rows)

0


#### Count and Replace player positions
If the player positions in the club is RES or SUB then his position will be replaced by the player's first position value

In [10]:
# An array holding all positions features present in the dataframe
POSITIONS = [  'ls' , 'st' , 'rs'  , 'lw' , 'lf'
             , 'cf' , 'rf' , 'rw'  , 'lam', 'cam'
             , 'ram', 'lm' , 'lcm' , 'cm' , 'rcm'
             , 'rm' , 'lwb', 'ldm' , 'cdm', 'rdm'
             , 'rwb', 'lb' , 'lcb' , 'cb' , 'rcb'
             , 'rb' , 'gk']

In [11]:
# Spliting information in cells on + or -
def split_position_skill(position_skill):
    parts = position_skill.split('+') if '+' in position_skill else position_skill.split('-') if '-' in position_skill else position_skill.split()
    general = int(parts[0])
    variation = 0 if len(parts) == 1 else int(parts[1]) if '+' in position_skill else -int(parts[1])
    return general, variation

In [12]:
%%timeit

# Convert str to int
for pos in POSITIONS:
    try:
        fifa[[pos, f'variation_{pos}']] = fifa[pos].apply(split_position_skill).apply(pd.Series)
        fifa[pos] = fifa[pos] + fifa[f'variation_{pos}']
        fifa.drop([f'variation_{pos}'], axis=1, inplace = True)
    except:
        break

<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

769 µs ± 166 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [13]:
# Creating a function to handle players positions feature
# present in the dataset
def player_position(data2):
    data2['player_positions'] = data2['player_positions'].str.split(',')
    sub_res_idx = data2.loc[(data2['club_position'] == 'SUB') | (data2['club_position'] == 'RES')].index
    sub_res_new_positions = data2.loc[sub_res_idx]['player_positions'].str[0]
    data2.loc[sub_res_idx, 'club_position'] = sub_res_new_positions
    data2['player_positions_count'] = data2['player_positions'].str.len()
    data2['player_positions_count'] = data2['player_positions'].str.len()
    print(data2[['short_name', 'player_positions_count']].head())
    data2[['short_name', 'player_positions_count']].head()

In [14]:
#Invoking player_positions with fifa
player_position(fifa)

          short_name  player_positions_count
0           L. Messi                       3
1     R. Lewandowski                       1
2  Cristiano Ronaldo                       2
3          Neymar Jr                       2
4       K. De Bruyne                       2


### Encoding relevent categorical attributes

In [15]:
# Encoding relevent categorical attributes
def encoding(data3):
    work_rates = {'Low/Low':0,'Medium/Low':1,'Low/Medium':1,'Medium/Medium':2,'High/Low':3,'Low/High':3,'High/Medium':4,'Medium/High':4,'High/High':5}
    data3['work_rate'] = data3['work_rate'].map(work_rates)

    preferd_foot = {'Left':0,'Right':1}
    data3['preferred_foot'] = data3['preferred_foot'].map(preferd_foot)

    body_types = {'Normal (170-)':0,'Normal (185+)':0,'Normal (170-185)':0,'Lean (170-)':1,'Lean (185+)':1,'Lean (170-185)':1,'Stocky (170-)':3,'Stocky (185+)':3,'Stocky (170-185)':3, 'Unique':5}
    data3['body_type'] = data3['body_type'].map(body_types) 

In [16]:
# Invoking encoding with fifa
encoding(fifa)

In [17]:
#Testing to confirm the  encoding
fifa['preferred_foot']

0        0
1        1
2        1
3        1
4        1
        ..
19234    1
19235    1
19236    1
19237    1
19238    1
Name: preferred_foot, Length: 19239, dtype: int64

In [18]:
#feature extraction process
corr_matrix = fifa.corr()

/tmp/ipykernel_31283/1564082060.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = fifa.corr()


In [19]:
# Sorting in descending order the correlation 
# between overall and other features
sortValues = corr_matrix['overall'].sort_values(ascending=False)
print(sortValues)

overall                1.000000
movement_reactions     0.871823
mentality_composure    0.708867
passing                0.663519
potential              0.644275
                         ...   
nationality_id        -0.168315
league_level          -0.178805
club_jersey_number    -0.185930
club_team_id          -0.226353
sofifa_id             -0.491523
Name: overall, Length: 88, dtype: float64


#### 2. Create feature subsets which show maximum correlation with the dependent variable.

In [20]:
# Selecting features with 
# absolute correlation greater than 0.4 [threshold correlation]
high_corr_data = sortValues[sortValues > abs(0.45)]

In [21]:
# select columns with correlation greater than the threshold
# select columns with correlation greater than the threshold
print(high_corr_data)

overall                     1.000000
movement_reactions          0.871823
mentality_composure         0.708867
passing                     0.663519
potential                   0.644275
dribbling                   0.618419
wage_eur                    0.601001
power_shot_power            0.561180
value_eur                   0.553823
lcm                         0.545862
rcm                         0.545862
cm                          0.545862
release_clause_eur          0.532528
mentality_vision            0.523025
attacking_short_passing     0.516788
skill_long_passing          0.498216
ram                         0.494720
lam                         0.494720
cam                         0.494720
physic                      0.491127
ls                          0.488558
st                          0.488558
rs                          0.488558
rdm                         0.482607
cdm                         0.482607
ldm                         0.482607
lf                          0.482157
c

### 4. Create and train a suitable machine learning model that can predict a player rating.

### Training the model

In [22]:
#Extracting features with higher correlation
features =  high_corr_data.index.tolist()
features

['overall',
 'movement_reactions',
 'mentality_composure',
 'passing',
 'potential',
 'dribbling',
 'wage_eur',
 'power_shot_power',
 'value_eur',
 'lcm',
 'rcm',
 'cm',
 'release_clause_eur',
 'mentality_vision',
 'attacking_short_passing',
 'skill_long_passing',
 'ram',
 'lam',
 'cam',
 'physic',
 'ls',
 'st',
 'rs',
 'rdm',
 'cdm',
 'ldm',
 'lf',
 'cf',
 'rf',
 'lm',
 'rm',
 'international_reputation',
 'skill_ball_control',
 'age',
 'rwb',
 'lwb',
 'rw',
 'lw',
 'shooting']

In [23]:
#Creating a new datasset with selected features
trainingData = fifa[features]
trainingData.head()

,overall,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,lcm,...,lm,rm,international_reputation,skill_ball_control,age,rwb,lwb,rw,lw,shooting
0,93,94,96,91.0,93,95.0,320000.0,86,78000000.0,90,...,93,93,5,96,34,69,69,92,92,92.0
1,92,93,88,79.0,92,86.0,270000.0,90,119500000.0,83,...,87,87,5,88,32,67,67,85,85,92.0
2,91,94,95,80.0,91,88.0,270000.0,94,45000000.0,81,...,89,89,5,88,36,66,66,88,88,94.0
3,91,89,93,86.0,91,94.0,270000.0,80,129000000.0,85,...,91,91,5,95,29,70,70,90,90,83.0
4,91,91,89,93.0,91,88.0,350000.0,91,125500000.0,91,...,91,91,4,91,30,82,82,88,88,86.0


In [24]:
# Dropping the dependent varibale - overall
selected_features = trainingData.drop('overall', axis=1)
selected_features

,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,lcm,rcm,...,lm,rm,international_reputation,skill_ball_control,age,rwb,lwb,rw,lw,shooting
0,94,96,91.0,93,95.0,320000.0,86,78000000.0,90,90,...,93,93,5,96,34,69,69,92,92,92.0
1,93,88,79.0,92,86.0,270000.0,90,119500000.0,83,83,...,87,87,5,88,32,67,67,85,85,92.0
2,94,95,80.0,91,88.0,270000.0,94,45000000.0,81,81,...,89,89,5,88,36,66,66,88,88,94.0
3,89,93,86.0,91,94.0,270000.0,80,129000000.0,85,85,...,91,91,5,95,29,70,70,90,90,83.0
4,91,89,93.0,91,88.0,350000.0,91,125500000.0,91,91,...,91,91,4,91,30,82,82,88,88,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,53,37,46.0,52,48.0,1000.0,46,70000.0,48,48,...,49,49,1,49,22,50,50,46,46,35.0
19235,49,47,50.0,59,46.0,500.0,49,110000.0,49,49,...,51,51,1,42,19,51,51,47,47,39.0
19236,46,36,45.0,55,49.0,500.0,50,100000.0,49,49,...,50,50,1,49,21,49,49,46,46,37.0
19237,48,47,36.0,60,48.0,500.0,48,110000.0,41,41,...,47,47,1,45,19,35,35,47,47,46.0


In [25]:
# Choose target and features
y = trainingData.overall
X = selected_features

In [26]:
# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
predicted_rating = forest_model.predict(val_X)
print(predicted_rating)

import sklearn.metrics as sm

mae = round(sm.mean_absolute_error(val_y, predicted_rating), 2)
mean_square = round(sm.mean_squared_error(val_y, predicted_rating), 2)
print("Mean absolute error =", mae)
print("Mean squared error =", mean_square)


[62.   65.   73.94 ... 71.98 67.98 69.49]
Mean absolute error = 0.28
Mean squared error = 0.25


#  Optimisation

In [28]:
from sklearn.feature_selection import SelectKBest, chi2
# load data
import sklearn.metrics as sm

def get_selector(k, X, y):
# select the 10 best features using chi-squared test
    selector = SelectKBest(chi2, k=k)
    X_new = selector.fit_transform(X, y)

    # get the indices of the selected features
#     selected_features1 = selected_features.columns[selector.get_support(indices=True)]
    selected_cols = X.columns[selector.get_support()]
    X_selected = selector.transform(X)
    df_selected = pd.DataFrame(X_selected, columns=selected_cols)

    #print(selected_features1)
    y = trainingData.overall
    X = df_selected
    from sklearn.model_selection import train_test_split
    train1_X, val1_X, train1_y, val1_y = train_test_split(X, y,random_state = 0)
    forest_model = RandomForestRegressor(random_state=1)
    forest_model.fit(train1_X, train1_y)
    predicted_rating1 = forest_model.predict(val1_X)
    print(predicted_rating1)
    
    mae = round(sm.mean_absolute_error(val_y, predicted_rating), 2)
# Check accuracy score 
   
    return mae

    #print('Model accuracy score: {0:0.4f}'. format(accuracy_score(val1_y, predicted_rating1)))

In [29]:
#Looping through different values of k
# to deduce how the accuracy is affected
for k in [2,5, 8,10, 13, 15, 17, 20, 21, 30, 35, 38]:
    get_selector1 = get_selector(k, X, y)
    print("Number of selectors: " + str(k) + " Mean absolute error: " + str(get_selector1))

[59.54249636 70.04744846 74.85722727 ... 67.27249094 64.78276711
 72.7895    ]
Number of selectors: 2 Mean absolute error: 0.28
[58.48 65.01 73.62 ... 64.86 67.88 70.04]
Number of selectors: 5 Mean absolute error: 0.28
[62.72 65.31 72.8  ... 68.17 68.84 69.85]
Number of selectors: 8 Mean absolute error: 0.28
[63.41 65.09 72.41 ... 67.32 69.12 69.75]
Number of selectors: 10 Mean absolute error: 0.28
[62.99 64.98 72.62 ... 67.37 69.31 69.77]
Number of selectors: 13 Mean absolute error: 0.28
[62.65 65.79 72.92 ... 67.19 69.52 69.63]
Number of selectors: 15 Mean absolute error: 0.28
[62.53 65.69 72.87 ... 67.3  69.63 69.58]
Number of selectors: 17 Mean absolute error: 0.28
[62.7  65.15 73.36 ... 68.11 68.32 69.36]
Number of selectors: 20 Mean absolute error: 0.28
[62.72 65.16 73.34 ... 68.26 68.24 69.37]
Number of selectors: 21 Mean absolute error: 0.28
[62.2  65.28 73.35 ... 67.96 68.11 68.91]
Number of selectors: 30 Mean absolute error: 0.28
[62.19 65.23 73.28 ... 67.93 68.08 68.87]
Numb

### Fine Turning

In [30]:
def get_mae(n_estimators, train_X, val_X, train_y, val_y):
    forest_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=25, max_features=30, random_state=1)
    forest_model.fit(train_X, train_y)
    predicted_rating = forest_model.predict(val_X)
    print(predicted_rating)
    mae = sm.mean_absolute_error(val_y, predicted_rating)
    #print(mae)
    return mae

In [45]:
import joblib
joblib.dump(forest_model,'rf_model.sav')

['rf_model.sav']

In [31]:
# compare MAE with differing values of max_leaf_nodes
for n_estimators in [215, 300]:
    my_mae = get_mae(n_estimators, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: " + str(n_estimators) + " Mean absolute error: " + str(my_mae))
    #print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(n_estimators, my_mae))

[61.95813953 64.98604651 73.8744186  ... 71.86511628 67.98139535
 69.47906977]
Max leaf nodes: 215 Mean absolute error: 0.28559502973456463
[61.95666667 64.98666667 73.87333333 ... 71.84333333 67.97333333
 69.5       ]
Max leaf nodes: 300 Mean absolute error: 0.2856203049203047


### 5. Use the data from another season which was not used during the training to test how good the model is.

In [32]:
# Creating a dataframe of 2021 
testing_data = pd.read_csv('players_22.csv')

/tmp/ipykernel_31283/2311378527.py:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_data = pd.read_csv('players_22.csv')


In [33]:
# Printing the first 5 entries in the dataset 
# to see how the dataset looks like
testing_data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [34]:
# Dropping all columns with  missing values greater than 60
missingData(testing_data)

club_loaned_from - 94%
nation_team_id - 96%
nation_position - 96%
nation_jersey_number - 96%
player_tags - 93%
goalkeeping_speed - 89%
nation_logo_url - 96%
Columns dropped :  

 ['club_loaned_from', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'player_tags', 'goalkeeping_speed', 'nation_logo_url']


In [35]:
# Imouting all the missing cells
replace_missing_values(testing_data)

In [36]:
# Get the count of missing values per row
missing_per_row = testing_data.isnull().sum(axis=1)

# Get the total number of rows with missing values
total_missing_rows = len(missing_per_row[missing_per_row > 0])

print(total_missing_rows)

0


In [37]:
# An array psostion relevant to the model testing
POSITIONS = [
 'lcm',
 'rcm',
 'cm',
 'ram',
 'lam',
 'cam',
 'ls',
 'st',
 'rs',
 'rdm',
 'cdm',
 'ldm',
 'lf',
 'cf',
 'rf',
 'lm',
 'rm',
 'rwb',
 'lwb',
 'rw',
 'lw',
]

In [38]:
# Handling players positions feature
# present in the dataset
def split_position(position_skill):
    parts = position_skill.split('+') if '+' in position_skill else position_skill.split('-') if '-' in position_skill else position_skill.split()
    general = int(parts[0])
    variation = 0 if len(parts) == 1 else int(parts[1]) if '+' in position_skill else -int(parts[1])
    return general, variation

In [39]:
%%timeit

# Convert str to int
for pos in POSITIONS:
    try:
        testing_data[[pos, f'variation_{pos}']] = testing_data[pos].apply(split_position).apply(pd.Series)
        testing_data[pos] = testing_data[pos] + testing_data[f'variation_{pos}']
        testing_data.drop([f'variation_{pos}'], axis=1, inplace = True)
    except:
        break

<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

855 µs ± 169 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [40]:
# Encoding the players positions
player_position(testing_data)

          short_name  player_positions_count
0           L. Messi                       3
1     R. Lewandowski                       1
2  Cristiano Ronaldo                       2
3          Neymar Jr                       2
4       K. De Bruyne                       2


In [41]:
# Encoding relevent categorical attributes
encoding(testing_data)

In [42]:
#Creating a new datasset with selected features
testingData = testing_data[['movement_reactions',
 'mentality_composure',
 'passing',
 'potential',
 'dribbling',
 'wage_eur',
 'power_shot_power',
 'value_eur',
 'lcm',
 'rcm',
 'cm',
 'release_clause_eur',
 'mentality_vision',
 'attacking_short_passing',
 'skill_long_passing',
 'ram',
 'lam',
 'cam',
 'physic',
 'ls',
 'st',
 'rs',
 'rdm',
 'cdm',
 'ldm',
 'lf',
 'cf',
 'rf',
 'lm',
 'rm',
 'international_reputation',
 'skill_ball_control',
 'age',
 'rwb',
 'lwb',
 'rw',
 'lw',
 'shooting']]
testingData.head()

,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,lcm,rcm,...,lm,rm,international_reputation,skill_ball_control,age,rwb,lwb,rw,lw,shooting
0,94,96,91.0,93,95.0,320000.0,86,78000000.0,90,90,...,93,93,5,96,34,69,69,92,92,92.0
1,93,88,79.0,92,86.0,270000.0,90,119500000.0,83,83,...,87,87,5,88,32,67,67,85,85,92.0
2,94,95,80.0,91,88.0,270000.0,94,45000000.0,81,81,...,89,89,5,88,36,66,66,88,88,94.0
3,89,93,86.0,91,94.0,270000.0,80,129000000.0,85,85,...,91,91,5,95,29,70,70,90,90,83.0
4,91,89,93.0,91,88.0,350000.0,91,125500000.0,91,91,...,91,91,4,91,30,82,82,88,88,86.0


In [43]:
#Predictions for the new data
prediction = forest_model.predict(testingData)

In [44]:
prediction

array([92.1 , 91.2 , 90.1 , ..., 47.49, 48.22, 47.62])